# This is an experimentation for the course project, focusing on the "substance use" and "mental health" aspects of the CSCS dataset

## Load in data

In [531]:
import pandas as pd
import plotly.express as px

pd.options.display.max_rows = 10

# Load data
cscs = pd.read_csv('CSCS_data_anon.csv')
cscs.head()

/var/folders/3c/3vwv00w96dz2b2bzgplw8fym0000gn/T/ipykernel_84647/1061607713.py:7: DtypeWarning:

Columns (408,1001,1002,1006,1007,1008,1080,1113,1115,1116,1117,1118,1119,1120,1121,1124,1125,1126,1127,1128,1213,1214,1215,1216,1217,1218,1342,1343,1344,1345,1346,1347,1348,1349,1390,1391,1393,1463,1549,1552,1555,1558,1561) have mixed types. Specify dtype option on import or set low_memory=False.



,UNIQUE_id,UNIQUE_num_records,ELIGIBLE_consent,GEO_residence_canada,GEO_province,DEMO_age,DEMO_gender,DEMO_identity_vetrans,DEMO_identity_indigenous,DEMO_identity_lgbtq,...,PSYCH_body_self_image_questionnaire_height_dissatisfaction_score,PSYCH_body_self_image_questionnaire_fatness_evaluation_score,PSYCH_body_self_image_questionnaire_negative_affect_score,PSYCH_body_self_image_questionnaire_social_dependence_score,PSYCH_big_five_inventory_agreeable_score,PSYCH_big_five_inventory_conscientious_score,PSYCH_big_five_inventory_extraverted_score,PSYCH_big_five_inventory_neurotic_score,PSYCH_big_five_inventory_open_score,REMOVE_case
0,cscs_00001,1,Yes,Yes,British Columbia,71.0,Non-binary,NaN,NaN,"Sexual or gender minorities (e.g., LGBTQ2+)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,cscs_00002,1,Yes,Yes,Ontario,69.0,Woman,NaN,NaN,Not Selected,...,3.0,8.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,No
2,cscs_00003,1,Yes,Yes,Quebec,56.0,Woman,NaN,NaN,Not Selected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,cscs_00005,1,Yes,Yes,NaN,54.0,Woman,NaN,NaN,Not Selected,...,NaN,NaN,NaN,NaN,28.0,34.0,30.0,32.0,37.0,No
4,cscs_00006,1,Yes,Yes,Ontario,30.0,Man,Not Selected,"Indigenous peoples (e.g., First Nations, Métis...","Sexual or gender minorities (e.g., LGBTQ2+)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


## Filter out columns related to SUBSTANCE_USE and get all unique values

In [532]:
cscs_substance = cscs.filter(like='SUBSTANCE_USE_drugs_', axis=1)
cscs_substance['SUBSTANCE_USE_drugs_alcohol'].unique()

array([nan, 'A few times a week', 'Less than monthly', 'Weekly',
       'A few times a month', 'Monthly', 'Not in the past six months',
       'Daily or almost daily', 'Presented but no response'], dtype=object)

## Map levels of substance use strings to int values of 0 or 1

In [533]:
import pandas as pd

mapping = {
    float('nan'): 0,  # Handles NaN values
    'Not in the past six months': 0,
    'Presented but no response': 0,
    'A few times a week': 1,
    'Less than monthly': 1,
    'Weekly': 1,
    'A few times a month': 1,
    'Monthly': 1,
    'Daily or almost daily': 1
}

cscs_substance = cscs_substance.replace(mapping).fillna(0).astype(int)
cscs_substance_valued = cscs_substance
cscs_substance_sum = cscs_substance_valued.sum()

# remove SUBSTANCE_USE_drugs_ prefix
cscs_substance_sum.index = cscs_substance_sum.index.str.replace('SUBSTANCE_USE_drugs_', '')

fig = px.bar(
    x=cscs_substance_sum.index,
    y=cscs_substance_sum.values,
    labels={'x': 'Columns', 'y': 'Count of True Values'},
    title='Count of True Values in Each Column'
)

# make labels angle
fig.update_xaxes(tickangle=45)
fig.show()

/var/folders/3c/3vwv00w96dz2b2bzgplw8fym0000gn/T/ipykernel_84647/49338326.py:15: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



## Sum levels of substance use by rows

In [534]:
cscs_substance_sum_rows = cscs_substance_valued.sum(axis=1)
cscs_substance_sum_rows

0        0
1        0
2        0
3        0
4        3
        ..
11426    0
11427    0
11428    0
11429    0
11430    0
Length: 11431, dtype: int64

## Remove substance use levels of 0 and show a freq graph for each level of usage

In [535]:
cscs_substance_sum_rows_nozero = cscs_substance_sum_rows.drop(cscs_substance_sum_rows[cscs_substance_sum_rows == 0].index)

fig_substance_freq = px.histogram(cscs_substance_sum_rows_nozero, title='Frequency of Substance Use')
fig_substance_freq.update_xaxes(title='Number of Substances Used')
fig_substance_freq.update_yaxes(title='Count')
fig_substance_freq.update_layout(showlegend=False)
fig_substance_freq.show()

## Take out columns for social anxiety Q18

In [536]:
cscs_anxiety = cscs.filter(like='PSYCH_social_interactions_anxiety_scale_', axis=1)
cscs_anxiety_copy = cscs.filter(like='PSYCH_social_interactions_anxiety_scale_', axis=1)
cscs_anxiety

,PSYCH_social_interactions_anxiety_scale_eye_contact,PSYCH_social_interactions_anxiety_scale_mixing,PSYCH_social_interactions_anxiety_scale_aquiantance,PSYCH_social_interactions_anxiety_scale_one_on_one,PSYCH_social_interactions_anxiety_scale_talking,PSYCH_social_interactions_anxiety_scale_disagree,PSYCH_social_interactions_anxiety_scale_score,PSYCH_social_interactions_anxiety_scale_score_y_n
0,Not at all characteristic or true of me,Slightly,Slightly,Slightly,Slightly,Not at all characteristic or true of me,4.0,No (0-6)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Moderately,2.0,No (0-6)
4,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Slightly,Not at all characteristic or true of me,Not at all characteristic or true of me,1.0,No (0-6)
...,...,...,...,...,...,...,...,...
11426,Not at all characteristic or true of me,Slightly,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,1.0,No (0-6)
11427,Slightly,Not at all characteristic or true of me,Slightly,Not at all characteristic or true of me,Slightly,Slightly,4.0,No (0-6)
11428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## oops, there seems to be an extra two columns

In [537]:
cscs_anxiety = cscs_anxiety.drop(columns=['PSYCH_social_interactions_anxiety_scale_score', 'PSYCH_social_interactions_anxiety_scale_score_y_n'])
cscs_anxiety

,PSYCH_social_interactions_anxiety_scale_eye_contact,PSYCH_social_interactions_anxiety_scale_mixing,PSYCH_social_interactions_anxiety_scale_aquiantance,PSYCH_social_interactions_anxiety_scale_one_on_one,PSYCH_social_interactions_anxiety_scale_talking,PSYCH_social_interactions_anxiety_scale_disagree
0,Not at all characteristic or true of me,Slightly,Slightly,Slightly,Slightly,Not at all characteristic or true of me
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Moderately
4,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Slightly,Not at all characteristic or true of me,Not at all characteristic or true of me
...,...,...,...,...,...,...
11426,Not at all characteristic or true of me,Slightly,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me,Not at all characteristic or true of me
11427,Slightly,Not at all characteristic or true of me,Slightly,Not at all characteristic or true of me,Slightly,Slightly
11428,NaN,NaN,NaN,NaN,NaN,NaN
11429,NaN,NaN,NaN,NaN,NaN,NaN


In [538]:
cscs_anxiety['PSYCH_social_interactions_anxiety_scale_eye_contact'].unique()

array(['Not at all characteristic or true of me', nan, 'Moderately',
       'Slightly', 'Very', 'Extremely characteristic or true of me',
       'Presented but no response'], dtype=object)

## Again, map the anxiety levels with corresponding values of ints (0-4)

In [539]:
mapping = {
    'Not at all characteristic or true of me': 0,
    float('nan'): 0,  # Handles NaN values
    'Presented but no response': 0,
    'Slightly': 1,
    'Moderately': 2,
    'Very': 3,
    'Extremely characteristic or true of me': 4
}

cscs_anxiety_valued = cscs_anxiety.replace(mapping).fillna(0).astype(int)
cscs_anxiety_valued

/var/folders/3c/3vwv00w96dz2b2bzgplw8fym0000gn/T/ipykernel_84647/751592285.py:11: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,PSYCH_social_interactions_anxiety_scale_eye_contact,PSYCH_social_interactions_anxiety_scale_mixing,PSYCH_social_interactions_anxiety_scale_aquiantance,PSYCH_social_interactions_anxiety_scale_one_on_one,PSYCH_social_interactions_anxiety_scale_talking,PSYCH_social_interactions_anxiety_scale_disagree
0,0,1,1,1,1,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,2
4,0,0,0,1,0,0
...,...,...,...,...,...,...
11426,0,1,0,0,0,0
11427,1,0,1,0,1,1
11428,0,0,0,0,0,0
11429,0,0,0,0,0,0


## Ok, let's try to filter only those with the most extreme substance uses (13) and see how they're doing mentally

In [540]:
cscs_anxiety_valued_extreme = cscs_anxiety_valued.drop(cscs_substance_sum_rows[cscs_substance_sum_rows != 13].index, axis=0)
cscs_anxiety_valued_extreme

,PSYCH_social_interactions_anxiety_scale_eye_contact,PSYCH_social_interactions_anxiety_scale_mixing,PSYCH_social_interactions_anxiety_scale_aquiantance,PSYCH_social_interactions_anxiety_scale_one_on_one,PSYCH_social_interactions_anxiety_scale_talking,PSYCH_social_interactions_anxiety_scale_disagree
60,2,2,1,2,2,2
64,2,3,2,1,2,1
74,2,1,3,2,1,1
97,1,2,3,2,3,2
128,1,1,2,2,2,2
...,...,...,...,...,...,...
11294,0,0,0,0,0,0
11324,3,2,2,3,2,2
11393,2,2,2,2,2,1
11395,2,1,1,2,1,2


In [541]:
cscs_anxiety_valued_extreme_sum_row = cscs_anxiety_valued_extreme.sum(axis=1)
cscs_anxiety_valued_extreme_sum_row

60       11
64       11
74       10
97       13
128      10
         ..
11294     0
11324    14
11393    11
11395     9
11397    14
Length: 275, dtype: int64

## Average?

In [542]:
cscs_anxiety_level_avg = cscs_anxiety_valued_extreme_sum_row.mean()
cscs_anxiety_level_avg

np.float64(10.305454545454545)

## Ok, let's now do that for every level of substance use including none (0-13)

In [543]:
list_avg_anxiety = []
for x in range(0, 14):
    cscs_anxiety_valued_extreme = cscs_anxiety_valued.drop(cscs_substance_sum_rows[cscs_substance_sum_rows != x].index, axis=0)
    cscs_anxiety_valued_extreme_sum_row = cscs_anxiety_valued_extreme.sum(axis=1)
    cscs_anxiety_level_avg = cscs_anxiety_valued_extreme_sum_row.mean()
    list_avg_anxiety.append(cscs_anxiety_level_avg)
    
list_avg_anxiety

[np.float64(2.5674376689696605),
 np.float64(4.24198250728863),
 np.float64(5.160839160839161),
 np.float64(5.381443298969073),
 np.float64(7.694117647058824),
 np.float64(7.4),
 np.float64(8.305555555555555),
 np.float64(6.782608695652174),
 np.float64(8.944444444444445),
 np.float64(8.130434782608695),
 np.float64(8.545454545454545),
 np.float64(8.88),
 np.float64(9.703125),
 np.float64(10.305454545454545)]

## Now let's map these averages as Y, and the level of substance use as X

In [544]:
x = list(range(0, 14))
y = list_avg_anxiety

fig1 = px.line(x=x, y=y, title='Average Anxiety Level by Substance Use Frequency', 
              labels={'x': 'Substance Use Frequency', 'y': 'Average Anxiety Level'})
# with scatter?
fig1.add_scatter(x=x, y=y, mode='markers', name='Substance Use <br>vs. <br>Anxiety Level')
fig1.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))
fig1.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.01
))
fig1.show()

In [545]:
cscs_anxiety_valued_sum = cscs_anxiety_valued.sum(axis=1)
cscs_anxiety_valued_sum

0        4
1        0
2        0
3        2
4        1
        ..
11426    1
11427    4
11428    0
11429    0
11430    0
Length: 11431, dtype: int64

## I wonder if I can fit a linear regression model through all the data points instead of directly averaging them out...

In [546]:
x = cscs_substance_sum_rows
y = cscs_anxiety_valued_sum

fig = px.scatter(x=x, y=y, title='Anxiety Level vs Substance Use Frequency', labels={'x': 'Substance Use Frequency', 'y': 'Anxiety Level'})
fig.show()

## ok.. that doesn't look great, but let's do it anyways.

In [547]:
import statsmodels.formula.api as smf

df = pd.DataFrame({'substance': cscs_substance_sum_rows, 'anxiety': cscs_anxiety_valued_sum})
model = smf.ols('anxiety ~ substance', data=df)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     1320.
Date:                Tue, 26 Nov 2024   Prob (F-statistic):          1.13e-273
Time:                        19:44:22   Log-Likelihood:                -33544.
No. Observations:               11431   AIC:                         6.709e+04
Df Residuals:                   11429   BIC:                         6.711e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.6612      0.044     60.242      0.000       2.575       2.748
substance      0.6387      0.018     36.336      0.000       0.604       0.673
==============================================================================
Omnibus:                     3608.839   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9256.344
Skew:                           1.748   Prob(JB):                         0.00
Kurtosis:                       5.686   Cond. No.                         2.64
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [548]:
# Draw the fitted SLR model line
import numpy as np

x = np.linspace(0, 13, 100)
y = results.params[0] + results.params[1] * x

fig = px.scatter(x=cscs_substance_sum_rows, y=cscs_anxiety_valued_sum, 
                 title='Anxiety Level vs Substance Use Frequency', 
                 labels={'x': 'Substance Use Frequency', 'y': 'Anxiety Level'})
fig.add_scatter(x=x, y=y, mode='lines', name='Fitted SLR Model')
fig.show()

/var/folders/3c/3vwv00w96dz2b2bzgplw8fym0000gn/T/ipykernel_84647/1286078150.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## I think lots of the data points are overlapping making them look weird, I'm going to draw this model on top of the original averaged scatterplot and see how that looks.

In [549]:
x = list(range(0, 14))
y = list_avg_anxiety

fig = px.line(x=x, y=y, title='Average Anxiety Level by Substance Use Frequency', 
              labels={'x': 'Substance Use Frequency', 'y': 'Average Anxiety Level'})

# add the fitted SLR model line
x = np.linspace(0, 13, 100)
y = results.params[0] + results.params[1] * x
fig1.add_scatter(x=x, y=y, mode='lines', name='Fitted SLR Model')
fig1.update_layout(legend=dict(
    yanchor="top",
    y=0.97,
    xanchor="left",
    x=0.006))
fig1.show()

/var/folders/3c/3vwv00w96dz2b2bzgplw8fym0000gn/T/ipykernel_84647/488150136.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## Great! This looks amazing.

## Ok, so those were levels of anxiety relating to social interactions, what about some others on the spectrum, such as burnout?

In [550]:
cscs_burnout = cscs.filter(like='WELLNESS_malach_pines_burnout_measure_', axis=1)
cscs_burnout

,WELLNESS_malach_pines_burnout_measure_tired,WELLNESS_malach_pines_burnout_measure_disappointed,WELLNESS_malach_pines_burnout_measure_hopeless,WELLNESS_malach_pines_burnout_measure_trapped,WELLNESS_malach_pines_burnout_measure_helpless,WELLNESS_malach_pines_burnout_measure_depressed,WELLNESS_malach_pines_burnout_measure_sick,WELLNESS_malach_pines_burnout_measure_worthless,WELLNESS_malach_pines_burnout_measure_difficulty_sleeping,WELLNESS_malach_pines_burnout_measure_had_it,WELLNESS_malach_pines_burnout_measure_score,WELLNESS_malach_pines_burnout_measure_score_y_n
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rarely,Never,Never,Almost never,Never,Sometimes,Rarely,Never,Sometimes,Rarely,2.3,No (1-3)
...,...,...,...,...,...,...,...,...,...,...,...,...
11426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11428,Often,Sometimes,Sometimes,Sometimes,Often,Often,Often,Sometimes,Always,Sometimes,4.7,Yes (4-7)
11429,Very Often,Sometimes,Sometimes,Sometimes,Sometimes,Often,Often,Often,Very Often,Sometimes,4.7,Yes (4-7)


## Hmm... I'm realizing that these categories all seem to have a "score" already calculated?

In [551]:
cscs_burnout_score = cscs_burnout['WELLNESS_malach_pines_burnout_measure_score']
cscs_burnout_score.count()

np.int64(4465)

In [552]:
fig = px.histogram(cscs_burnout_score, title='Burnout Score Distribution', labels={'value': 'Burnout Score'})
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.97,
    xanchor="left",
    x=0.006))
fig.show()

## Wow! Finally we see something of a normal distribution! This is huge. I'm concerned about that big weird bar at 1 though. Also Judging from the number of unique values for this column I don't know if this counts as continuous data. We'll see what we can do though.

## Let's see the same for anxiety score, I've made a copy of it before I removed the two columns:

In [553]:
cscs_anxiety_score = cscs_anxiety_copy['PSYCH_social_interactions_anxiety_scale_score'].dropna()
cscs_anxiety_score

0         4.0
3         2.0
4         1.0
6        11.0
8         1.0
         ... 
11422     8.0
11425    13.0
11426     1.0
11427     4.0
11430     0.0
Name: PSYCH_social_interactions_anxiety_scale_score, Length: 5361, dtype: float64

In [554]:
cscs_anxiety_score.unique()

array([ 4.,  2.,  1., 11.,  7.,  6.,  0.,  9.,  3., 14., 12., 18.,  5.,
       10.,  8., 13., 16., 15., 17., 21., 22., 24., 19., 20., 23.])

In [555]:
fig = px.histogram(cscs_anxiety_score, title='Anxiety Score Distribution', labels={'value': 'Anxiety Score'})
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.97,
    xanchor="right",
    x=0.993))
fig.show()

## Not quite of a normal distribution anymore, but I think it does make sense though.

## Now looking for more variables that are somewhat continuous (With lots of values and not just like 5 integers or something)

In [556]:
cscs_lonley = cscs.filter(like='LONELY_ucla_loneliness_scale_', axis=1)
cscs_lonley

,LONELY_ucla_loneliness_scale_companionship,LONELY_ucla_loneliness_scale_left_out,LONELY_ucla_loneliness_scale_isolated,LONELY_ucla_loneliness_scale_score,LONELY_ucla_loneliness_scale_score_y_n
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,Some of the time,Hardly Ever,Hardly Ever,4.0,No (3-5)
...,...,...,...,...,...
11426,NaN,NaN,NaN,NaN,NaN
11427,NaN,NaN,NaN,NaN,NaN
11428,Often,Some of the time,Some of the time,7.0,Yes (6-9)
11429,Some of the time,Hardly Ever,Some of the time,5.0,No (3-5)


In [557]:
cscs_lonley['LONELY_ucla_loneliness_scale_score'].unique()

array([nan,  4.,  9.,  5.,  6.,  7.,  8.,  3.])

In [558]:
cscs_dejong_lonely = cscs.filter(like='LONELY_dejong_emotional_social_loneliness_scale_', axis=1)
cscs_dejong_lonely

,LONELY_dejong_emotional_social_loneliness_scale_emptiness,LONELY_dejong_emotional_social_loneliness_scale_rely,LONELY_dejong_emotional_social_loneliness_scale_trust,LONELY_dejong_emotional_social_loneliness_scale_close,LONELY_dejong_emotional_social_loneliness_scale_miss,LONELY_dejong_emotional_social_loneliness_scale_rejected,LONELY_dejong_emotional_social_loneliness_scale_score,LONELY_dejong_emotional_social_loneliness_scale_score_y_n
0,Yes,No,More or less,No,Yes,No,5.0,Yes (2-6)
1,No,Yes,Yes,Yes,No,No,0.0,No (0-1)
2,No,Yes,Yes,More or less,Yes,No,2.0,Yes (2-6)
3,Yes,No,More or less,No,Yes,Yes,6.0,Yes (2-6)
4,No,No,No,Yes,Yes,No,3.0,Yes (2-6)
...,...,...,...,...,...,...,...,...
11426,No,More or less,No,More or less,Yes,No,4.0,Yes (2-6)
11427,No,Yes,Yes,Yes,No,No,0.0,No (0-1)
11428,More or less,No,No,Yes,Yes,No,4.0,Yes (2-6)
11429,No,No,No,No,Yes,No,4.0,Yes (2-6)


In [559]:
cscs_dejong_lonely['LONELY_dejong_emotional_social_loneliness_scale_score'].unique()

array([ 5.,  0.,  2.,  6.,  3.,  1.,  4., nan])

## Filter for all columns with "score" in them

In [560]:
cscs_score = cscs.filter(like='_score', axis=1)
column_list = cscs_score.columns
for column in column_list:
    print(column)

PSYCH_zimet_multidimensional_social_support_scale_positive_not_scored
PSYCH_zimet_multidimensional_social_support_scale_gets_me_not_scored
PSYCH_cope_60_positive_reinterpretation_and_growth_subscale_score
PSYCH_cope_60_mental_disengagement_subscale_score
PSYCH_cope_60_focus_on_and_venting_of_emotions_subscale_score
PSYCH_cope_60_use_of_instrumental_social_support_subscale_score
PSYCH_cope_60_active_coping_subscale_score
PSYCH_cope_60_denial_subscale_score
PSYCH_cope_60_religious_coping_subscale_score
PSYCH_cope_60_humor_subscale_score
PSYCH_cope_60_behavioral_disengagement_subscale_score
PSYCH_cope_60_restraint_subscale_score
PSYCH_cope_60_use_of_emotional_social_support_subscale_score
PSYCH_cope_60_substance_use_subscale_score
PSYCH_cope_60_acceptance_subscale_score
PSYCH_cope_60_suppression_of_competing_activities_subscale_score
PSYCH_cope_60_planning_subscale_score
HEALTH_hampson_good_health_practices_scale_score
PSYCH_struk_short_boredom_proneness_scale_score
SUBSTANCE_USE_dast_10_

In [561]:
cscs_score = cscs.filter(like='_score', axis=1)
cscs_score_lonely = cscs_score.filter(like='LONELY', axis=1)
cscs_score_lonely_score = cscs_score_lonely['LONELY_existential_loneliness_scale_score']
cscs_score_lonely_score

0         NaN
1         NaN
2         NaN
3         NaN
4        39.0
         ... 
11426     NaN
11427     NaN
11428    40.0
11429    29.0
11430    41.0
Name: LONELY_existential_loneliness_scale_score, Length: 11431, dtype: float64

In [562]:
cscs_score_lonely_score.unique()

array([nan, 39., 36., 23., 18., 13., 16., 31., 27., 35., 20., 28., 33.,
       26., 29., 25., 30., 41., 32., 47., 38., 37., 34., 50., 40., 54.,
       42., 49., 48., 21., 15., 17., 22., 24., 45., 44.,  6., 43., 46.,
       12., 14., 19., 52., 10.,  8., 11.,  9.,  7., 51.])

In [563]:
fig = px.histogram(cscs_score_lonely['LONELY_existential_loneliness_scale_score'], 
                   title='Existential Loneliness Score Distribution', 
                   labels={'value': 'Existential Loneliness Score'})
fig.update_layout(legend=dict(
                    yanchor="top",
                    y=0.97,
                    xanchor="left",
                    x=0.006))
fig.show()

## Good one!

In [564]:
cscs_connection = cscs.filter(like='CONNECTION_activities_', axis=1)
cscs_connection 


,CONNECTION_activities_talked_day_p3m,CONNECTION_activities_talked_family_p3m,CONNECTION_activities_talked_job_p3m,CONNECTION_activities_talked_hobbies_p3m,CONNECTION_activities_phone_p3m,CONNECTION_activities_letter_or_email_p3m,CONNECTION_activities_checked_in_p3m,CONNECTION_activities_text_or_messaged_p3m,CONNECTION_activities_chat_p3m,CONNECTION_activities_video_chat_p3m,...,CONNECTION_activities_visited_family_pm,CONNECTION_activities_visited_friends_pm,CONNECTION_activities_community_pm,CONNECTION_activities_walk_pm,CONNECTION_activities_games_p3m,CONNECTION_activities_games_last,CONNECTION_activities_games_pm,CONNECTION_activities_talked_last,CONNECTION_activities_talked_p3m,CONNECTION_activities_talked_pm
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,No,Yes,NaN,Not in the past three months,No,Earlier today,NaN,Yes
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,No,Yes,NaN,Earlier today,Yes,Earlier today,NaN,Yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,No,Yes,Yes,NaN,In the past week,Yes,In the past two or three days,NaN,Yes
3,NaN,NaN,NaN,NaN,Daily or almost daily,NaN,NaN,A few times a week,Daily or almost daily,NaN,...,Yes,Yes,Yes,Yes,Monthly,NaN,Yes,NaN,NaN,NaN
4,Weekly,A few times a month,Weekly,A few times a week,Daily or almost daily,Monthly,A few times a month,Monthly,Weekly,A few times a week,...,Yes,Yes,Yes,Yes,A few times a week,NaN,Yes,NaN,A few times a week,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,NaN,NaN,NaN,NaN,Daily or almost daily,NaN,NaN,Daily or almost daily,Daily or almost daily,NaN,...,Yes,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN
11427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,No,Yes,NaN,Earlier today,Yes,Earlier today,NaN,Yes
11428,Weekly,Weekly,A few times a week,Weekly,Daily or almost daily,Not in the past three months,A few times a month,Daily or almost daily,Daily or almost daily,A few times a week,...,Yes,No,No,No,Not in the past three months,NaN,No,NaN,A few times a week,Yes
11429,A few times a week,Weekly,A few times a month,A few times a month,A few times a month,Not in the past three months,A few times a month,Weekly,Daily or almost daily,Daily or almost daily,...,Yes,No,No,No,Not in the past three months,NaN,No,NaN,A few times a week,Yes


## Attempt for doing bootstrapping on burnout and performing null hypothesis with bootstrapped means:

In [565]:
cscs_burnout_score_has_substance = cscs_burnout_score.drop(cscs_substance_sum_rows[cscs_substance_sum_rows <= 7].index)
cscs_burnout_score_has_substance

39       2.1
60       4.7
64       3.9
74       3.8
83       3.8
        ... 
11324    3.9
11326    3.5
11393    2.6
11395    3.8
11397    4.3
Name: WELLNESS_malach_pines_burnout_measure_score, Length: 427, dtype: float64

In [566]:
# Horizontally sum
cscs_burnout_score_has_substance.sum()
cscs_burnout_score_has_substance_mean = cscs_burnout_score_has_substance.mean()

In [567]:
# Mark mean value on the original histogram
fig = px.histogram(cscs_burnout_score, title='Burnout Score Distribution', labels={'value': 'Burnout Score'})
fig.add_vline(x=cscs_burnout_score_has_substance_mean, 
              line_dash='dash', line_color='red', 
              annotation_text='Mean With Substance Use', 
              annotation_position='top right')
fig.update_layout(showlegend=False)
fig.show()

In [568]:
# Now also mark the mean of the original data
cscs_burnout_score_mean = cscs_burnout_score.mean()
fig.add_vline(x=cscs_burnout_score_mean, line_dash='dash', 
              line_color='green', annotation_text='Real Mean', 
              annotation_position='top left')
fig.show()

## This is a little concerning... We don't really see a big difference between the Real mean and the Mean for recipients with substance use... What else can we do?

In [569]:
# Histogram of burnout score for those who have substance use
fig = px.histogram(cscs_burnout_score_has_substance, 
                   title='Burnout Score Distribution for Those Who Have Substance Use', 
                   labels={'value': 'Burnout Score'})
fig.update(layout_showlegend=False)
fig.show()

## Let's try bootstrapping anyways

In [570]:
# Perform bootstrapping on the original burnout score, calculate the mean, and plot the histogram
import numpy as np

n = 1000
mean_list = []
for i in range(n):
    sample = cscs_burnout_score.sample(frac=1, replace=True)
    mean_list.append(sample.mean())

fig = px.histogram(mean_list, title='Bootstrapped Burnout Score Mean Distribution')
fig.update(layout_showlegend=False)
fig.show()


In [571]:
# Mark the mean of the burnout score for those who have substance use
fig = px.histogram(mean_list, title='Bootstrapped Burnout Score Mean Distribution', 
                   labels={'value': 'Mean Burnout Score'})
fig.add_vline(x=cscs_burnout_score_has_substance_mean, line_dash='dash', 
              line_color='red', annotation_text='Mean With Substance Use', 
              annotation_position='top right')
fig.update(layout_showlegend=False)
fig.show()


In [572]:
# Mark 95% confidence interval of the bootstrapped mean
lower_bound = np.percentile(mean_list, 2.5)
upper_bound = np.percentile(mean_list, 97.5)
fig.add_vline(x=lower_bound, line_dash='dash', line_color='blue', 
              annotation_text='95% CI Lower Bound', annotation_position='top left')
fig.add_vline(x=upper_bound, line_dash='dash', line_color='blue', 
              annotation_text='95% CI Upper Bound', annotation_position='top right')
fig.show()

## What the fuck ok cool this is totally statistically significant

## Let's try on the lonliness scale too

In [573]:
# Perform bootstrapping on the loneliness score, calculate the mean, and plot the histogram
cscs_lonely_score = cscs_score_lonely['LONELY_existential_loneliness_scale_score'].dropna()

n = 1000
mean_list = []
for i in range(n):
    sample = cscs_lonely_score.sample(frac=1, replace=True)
    mean_list.append(sample.mean())
    
fig = px.histogram(mean_list, 
                   title='Bootstrapped Loneliness Score Mean Distribution', 
                   labels={'value': 'Mean Loneliness Score'})
fig.update_layout(showlegend=False)
fig.show()

In [575]:
# Mark the mean of the loneliness score for those who have substance use
cscs_lonely_score_has_substance = cscs_score_lonely_score.drop(cscs_substance_sum_rows[cscs_substance_sum_rows <= 7].index)
cscs_lonely_score_has_substance_mean = cscs_lonely_score_has_substance.mean()

fig = px.histogram(mean_list, title='Bootstrapped Loneliness Score Mean Distribution', 
                   labels={'value': 'Mean Loneliness Score'})
fig.add_vline(x=cscs_lonely_score_has_substance_mean, line_dash='dash', 
              line_color='red', annotation_text='Mean With Substance Use', 
              annotation_position='top left')
fig.update_layout(showlegend=False)
fig.show()


In [576]:
# Mark 95% confidence interval of the bootstrapped mean
lower_bound = np.percentile(mean_list, 2.5)
upper_bound = np.percentile(mean_list, 97.5)
fig.add_vline(x=lower_bound, line_dash='dash', line_color='blue', annotation_text='95% CI Lower Bound', annotation_position='top left')
fig.add_vline(x=upper_bound, line_dash='dash', line_color='blue', annotation_text='95% CI Upper Bound', annotation_position='top right')
fig.show()